# KNN

Este notebook contem a implementação do $KNN$ força bruta feita para a disciplina CK0149 - GARIMPAGEM DE DADOS 2016.2

## 1. Handler dos dados

Primeiro, precisamos carregar os dados do arquivo do dataset e organizá-los em alguma estrutura. Teremos de usar o NumPy para a manipulação das matrizes.

In [1]:
import numpy as np
import math
import warnings
warnings.filterwarnings("ignore")

data = np.loadtxt("haberman.data", delimiter = ',')
len(data)

306

Agora, os dados do dataset estão armazenados na matriz data. Agora, devemos embaralhar os dados e separá-los em conjuntos de treino e teste. Como temos 306 entradas no dataset, usaremos uma porcentagem definida abaixo como 67% para treino e o resto para teste.

In [2]:
rand_data = np.random.permutation(data)

sep = int(0.67*len(rand_data))

train_data = rand_data[:sep, :3]
test_data = rand_data[sep:, :3]

yTrain_data = rand_data[:sep, 3:]
yTest_data = rand_data[sep:, 3:]


Agora os dados estão prontos para a manipulação.

## 2. Medida de Similaridade

O algoritmo $KNN$ busca o conjunto de $K$ pontos mais próximos de um ponto $x$. Assim, precisamos definir uma medida de distância para comparar os dados. Como nosso conjunto de dados é numérico, usaremos a distância euclidiana para tanto. Usando o NumPy, podemos fazer esse calculo da seguinte maneira

In [3]:
test =  np.linalg.norm(train_data[1] - train_data[2])
test

20.024984394500787

## 3. Vizinhos mais próximos

Agora precisamos computar para cada ponto os seus $K$ vizinhos mais próximos. Pra isso, definimos a função getNeighbours() que, dado um ponto, retorna uma lista com seus $K$ vizinhos mais próximos:

In [4]:
import operator
def getNeighbours(point, trainingData, k):
    distances = []
    length = len(point) - 1
    
    for p in range(len(trainingData)):
        if not np.array_equal(trainingData[p], point):
            distances.append([p, np.linalg.norm(point - trainingData[p])])
        else:
            distances.append([0, 0])
    distances.sort(key = operator.itemgetter(1))
    distances.pop(0)
    
    neighbours = []
    for i in range(k):
        neighbours.append(distances[i][0])
    return neighbours

## 4. Classificação

Uma vez que possuimos os vizinhos mais próximos, precisamos decidir a classe de cada elemento. Para isso, navegaremos pelos vizinhos mais próximos e contaremos a classe com mais representantes.

In [5]:
def classify(neighbours):
    classCounter = {}
    for n in neighbours:
        if yTrain_data[n][0] in classCounter:
            classCounter[yTrain_data[n][0]] += 1
        else:
            classCounter[yTrain_data[n][0]] = 1
    
    output = ''
    maxElement = 0
    for c in classCounter:
        if classCounter[c] > maxElement:
            maxElement = classCounter[c]
            output = c
    return output
            

classify(getNeighbours(train_data[0], train_data, 3))

1.0

## 5. Medida de precisão

Agora, uma vez que somos aptos a classificar conjuntos novos de dados, precisamos de uma função que diga o quão correto foi o processo de classificação. 

In [6]:
def accuracy(yTest, predictions):
    correct = 0.
    for i in range(len(yTest)):
        if int(yTest[i]) == int(predictions[i]):
            correct += 1
    return (correct/len(yTest))*100.0

## 6. Executando o algoritmo

Após todos esses passos, precisamos executar o algoritmo para o conjunto de teste e calcular a precisão.

In [7]:
predictions = []
for d in test_data:
    predictions.append(classify(getNeighbours(d, train_data, 3)))

accuracy(yTest_data, predictions)

79.20792079207921

## 7. Comparação com scikit-learn

Como requisitado na proposição do trabalho, agora iremos comparar os resultados obtidos na implementação acima com os do scikit-learn. Para isso, devemos importar as funções necessárias e aplicá-las para os mesmos conjuntos de treino e teste.

In [8]:
from sklearn.neighbors import KNeighborsClassifier

knn3 = KNeighborsClassifier(n_neighbors=3)
knn3.fit(train_data, yTrain_data)
knn3.score(test_data, yTest_data)

/opt/ipnb/lib/python2.7/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


0.75247524752475248

Note que os resultados foram muito similares. Nos testes executados, a variação entre os dois não passou de 5%.